# Enter State Farm

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"
path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
batch_size=64

## Create sample

The following assumes you've already created your validation set - remember that the training and validation set should contain *different drivers*, as mentioned on the Kaggle competition page.

In [4]:
%cd /home/ubuntu/courses/deeplearning1/data/state

/home/ubuntu/courses/deeplearning1/data/state


In [5]:
%cd train

/home/ubuntu/courses/deeplearning1/data/state/train


In [6]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

mkdir: cannot create directory ‘../sample’: File exists
mkdir: cannot create directory ‘../sample/train’: File exists
mkdir: cannot create directory ‘../sample/valid’: File exists


In [10]:
# This section creates the validation set.
%cd ..
directory = 'valid'
if not os.path.exists(directory):
    os.makedirs(directory)
cwd = os.getcwd()
# path = cwd + '/' + directory
path = "/home/ubuntu/courses/deeplearning1/data/state/valid"
[os.mkdir('{}/c{}'.format(path, i)) for i in range(10)]

/home/ubuntu/courses/deeplearning1/data


[None, None, None, None, None, None, None, None, None, None]

In [2]:
# read in the 'driver_imgs_list.csv' file
import pandas as pd
df = pd.read_csv("/home/ubuntu/courses/deeplearning1/data/state/driver_imgs_list.csv")

In [4]:
# There are 26 unique drivers in the train set. 
# We want 20%, so pick 5 at random
import random
driver_names = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
driver_names_for_validation = random.sample(driver_names, 5)

In [5]:
# filter df by selecting rows with the subjects we want
df = df[df['subject'].isin(driver_names_for_validation)]

In [7]:
#dftest = df.loc[0:1]
#    print str(i) + ': ' +  'moving ' + to_move

# do the move here:
import shutil
i = 0
for index, row in df.iterrows():
    to_move = '/home/ubuntu/courses/deeplearning1/data/state/train' + '/' +  row['classname'] + '/' + row['img']
    print to_move
    i = i + 1
    move_to = '/home/ubuntu/courses/deeplearning1/data/state/valid' + '/' +  row['classname']
    print move_to
    shutil.move(to_move, move_to)

/home/ubuntu/courses/deeplearning1/data/state/train/c0/img_58285.jpg
/home/ubuntu/courses/deeplearning1/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/data/state/train/c0/img_31808.jpg
/home/ubuntu/courses/deeplearning1/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/data/state/train/c0/img_34091.jpg
/home/ubuntu/courses/deeplearning1/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/data/state/train/c0/img_17767.jpg
/home/ubuntu/courses/deeplearning1/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/data/state/train/c0/img_2528.jpg
/home/ubuntu/courses/deeplearning1/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/data/state/train/c0/img_18895.jpg
/home/ubuntu/courses/deeplearning1/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/data/state/train/c0/img_24692.jpg
/home/ubuntu/courses/deeplearning1/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/data/state/train/c0/img_83000.jpg
/home/ubuntu/courses/deeplearning1/data/state/valid/c0
/home/ubu

In [8]:
print 'files to move: ' ,df.shape[0]
print 'files moved:   ' ,i

files to move:  5152
files moved:    5152


In [7]:
# Jeremy making the sample training and validation sets
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [8]:
from shutil import copyfile

In [9]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [10]:
%cd ../valid

/home/ubuntu/courses/deeplearning1/data/state/valid


In [11]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [12]:
%cd ../../..

/home/ubuntu/courses/deeplearning1


In [13]:
%mkdir data/state/results

In [14]:
%mkdir data/state/sample/test

## Create batches

In [15]:
path = "data/state/sample/"
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [16]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


## Basic models

### Linear model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.

In [17]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

As you can see below, this training is going nowhere...

In [18]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 35s - loss: 13.7192 - acc: 0.1093 - val_loss: 14.0545 - val_acc: 0.1270
Epoch 2/2
1500/1500 [==============================] - 26s - loss: 14.2806 - acc: 0.1140 - val_loss: 14.4585 - val_acc: 0.1030


Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [20]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNormal(None, 3, 224, 224)   6           batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_1[0][0]                  
Total params: 1505296
____________________________________________________________________________________________________


Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [21]:
10*3*224*224

1505280

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [22]:
np.round(model.predict_generator(batches, batches.N)[:10],2)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]], dtype=float32)

Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate:

In [23]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 2.4375 - acc: 0.1867 - val_loss: 4.6171 - val_acc: 0.1730
Epoch 2/2
1500/1500 [==============================] - 25s - loss: 1.7264 - acc: 0.4273 - val_loss: 2.7229 - val_acc: 0.2390


Great - we found our way out of that hole... Now we can increase the learning rate and see where we can get to.

In [24]:
model.optimizer.lr=0.001

In [25]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 33s - loss: 1.3323 - acc: 0.6033 - val_loss: 2.5416 - val_acc: 0.2550
Epoch 2/4
1500/1500 [==============================] - 25s - loss: 1.0822 - acc: 0.7227 - val_loss: 2.1054 - val_acc: 0.2940
Epoch 3/4
1500/1500 [==============================] - 26s - loss: 0.9017 - acc: 0.7927 - val_loss: 1.9747 - val_acc: 0.3330
Epoch 4/4
1500/1500 [==============================] - 24s - loss: 0.7771 - acc: 0.8413 - val_loss: 1.9376 - val_acc: 0.3640


We're stabilizing at validation accuracy of 0.39. Not great, but a lot better than random. Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [26]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 1000 images belonging to 10 classes.


In [27]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

array([[ 1.94,  0.36],
       [ 1.96,  0.36],
       [ 1.93,  0.37],
       [ 1.94,  0.37],
       [ 1.93,  0.37],
       [ 1.95,  0.36],
       [ 1.93,  0.37],
       [ 1.93,  0.37],
       [ 1.94,  0.36],
       [ 1.96,  0.36]])

Yup, pretty consistent - if we see improvements of 3% or more, it's probably not random, based on the above samples.

### L2 regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (i.e. add the sum of squares of the weights to our loss function):

In [28]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=l2(0.01))
    ])
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 5.1424 - acc: 0.3027 - val_loss: 8.4078 - val_acc: 0.1990
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 3.2303 - acc: 0.5873 - val_loss: 5.9598 - val_acc: 0.2950


In [29]:
model.optimizer.lr=0.001

In [30]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 32s - loss: 2.4803 - acc: 0.7440 - val_loss: 6.2852 - val_acc: 0.2630
Epoch 2/4
1500/1500 [==============================] - 24s - loss: 2.1108 - acc: 0.8327 - val_loss: 4.7998 - val_acc: 0.2980
Epoch 3/4
1500/1500 [==============================] - 25s - loss: 2.1508 - acc: 0.8667 - val_loss: 4.5717 - val_acc: 0.2930
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 1.8778 - acc: 0.8913 - val_loss: 4.5922 - val_acc: 0.3110


Looks like we can get a bit over 50% accuracy this way. This will be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [31]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 1.9088 - acc: 0.4073 - val_loss: 8.2236 - val_acc: 0.1350
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 0.9311 - acc: 0.7353 - val_loss: 4.5325 - val_acc: 0.1930
Epoch 1/5
1500/1500 [==============================] - 33s - loss: 0.5075 - acc: 0.8967 - val_loss: 2.3962 - val_acc: 0.3050
Epoch 2/5
1500/1500 [==============================] - 26s - loss: 0.3139 - acc: 0.9513 - val_loss: 1.9494 - val_acc: 0.3680
Epoch 3/5
1500/1500 [==============================] - 24s - loss: 0.1922 - acc: 0.9813 - val_loss: 2.2239 - val_acc: 0.3340
Epoch 4/5
1500/1500 [==============================] - 24s - loss: 0.1377 - acc: 0.9920 - val_loss: 1.8371 - val_acc: 0.3700
Epoch 5/5
1500/1500 [==============================] - 24s - loss: 0.0977 - acc: 0.9947 - val_loss: 2.0164 - val_acc: 0.3710


Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single conv layer

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [32]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [33]:
conv1(batches)

Exception in thread Thread-42:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/training.py", line 425, in data_generator_task
    generator_output = next(generator)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 600, in next
    img = load_img(os.path.join(self.directory, fname), grayscale=grayscale, target_size=self.target_size)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 173, in load_img
    img = Image.open(path)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/PIL/Image.py", line 2280, in open
    fp = builtins.open(filename, "rb")
IOError: [Errno 2] No such file or directory: 'data/state/

Epoch 1/2
1500/1500 [==============================] - 37s - loss: 1.6123 - acc: 0.5287 - val_loss: 2.9678 - val_acc: 0.1880
Epoch 2/2
1500/1500 [==============================] - 29s - loss: 0.3821 - acc: 0.9287 - val_loss: 2.4560 - val_acc: 0.1450
Epoch 1/4
1500/1500 [==============================] - 35s - loss: 0.1220 - acc: 0.9933 - val_loss: 2.5962 - val_acc: 0.1590
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 0.0589 - acc: 0.9987 - val_loss: 2.9331 - val_acc: 0.1510
Epoch 3/4
1500/1500 [==============================] - 28s - loss: 0.0337 - acc: 1.0000 - val_loss: 3.1343 - val_acc: 0.1240
Epoch 4/4
1500/1500 [==============================] - 27s - loss: 0.0195 - acc: 1.0000 - val_loss: 3.2558 - val_acc: 0.1210


The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable result.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

Width shift: move the image left and right -

In [34]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [35]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 34s - loss: 2.1310 - acc: 0.3413 - val_loss: 2.4801 - val_acc: 0.1640
Epoch 2/2
1500/1500 [==============================] - 31s - loss: 1.1357 - acc: 0.6580 - val_loss: 2.0133 - val_acc: 0.3440
Epoch 1/4
1500/1500 [==============================] - 38s - loss: 0.7317 - acc: 0.7907 - val_loss: 2.2434 - val_acc: 0.1630
Epoch 2/4
1500/1500 [==============================] - 28s - loss: 0.5525 - acc: 0.8493 - val_loss: 2.3504 - val_acc: 0.1430
Epoch 3/4
1500/1500 [==============================] - 28s - loss: 0.4210 - acc: 0.8940 - val_loss: 2.6301 - val_acc: 0.1280
Epoch 4/4
1500/1500 [==============================] - 30s - loss: 0.3869 - acc: 0.8907 - val_loss: 2.6822 - val_acc: 0.1620


Height shift: move the image up and down -

In [36]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [37]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 34s - loss: 1.8611 - acc: 0.4313 - val_loss: 2.8414 - val_acc: 0.1170
Epoch 2/2
1500/1500 [==============================] - 31s - loss: 0.7331 - acc: 0.8007 - val_loss: 2.4257 - val_acc: 0.2180
Epoch 1/4
1500/1500 [==============================] - 34s - loss: 0.3661 - acc: 0.9167 - val_loss: 2.6869 - val_acc: 0.1670
Epoch 2/4
1500/1500 [==============================] - 29s - loss: 0.2394 - acc: 0.9427 - val_loss: 2.8166 - val_acc: 0.1820
Epoch 3/4
1500/1500 [==============================] - 27s - loss: 0.1627 - acc: 0.9653 - val_loss: 2.8931 - val_acc: 0.1890
Epoch 4/4
1500/1500 [==============================] - 29s - loss: 0.1342 - acc: 0.9767 - val_loss: 2.8956 - val_acc: 0.2130


Random shear angles (max in radians) -

In [38]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [39]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 35s - loss: 1.6178 - acc: 0.5107 - val_loss: 2.3617 - val_acc: 0.1820
Epoch 2/2
1500/1500 [==============================] - 30s - loss: 0.4404 - acc: 0.8880 - val_loss: 2.2839 - val_acc: 0.1090
Epoch 1/4
1500/1500 [==============================] - 36s - loss: 0.1669 - acc: 0.9800 - val_loss: 2.3973 - val_acc: 0.1410
Epoch 2/4
1500/1500 [==============================] - 28s - loss: 0.1071 - acc: 0.9847 - val_loss: 2.7204 - val_acc: 0.1520
Epoch 3/4
1500/1500 [==============================] - 28s - loss: 0.0662 - acc: 0.9947 - val_loss: 2.8088 - val_acc: 0.1700
Epoch 4/4
1500/1500 [==============================] - 29s - loss: 0.0438 - acc: 0.9980 - val_loss: 2.8618 - val_acc: 0.1830


Rotation: max in degrees -

In [40]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [41]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 36s - loss: 2.1129 - acc: 0.3613 - val_loss: 2.4356 - val_acc: 0.1770
Epoch 2/2
1500/1500 [==============================] - 29s - loss: 0.9586 - acc: 0.7133 - val_loss: 2.0725 - val_acc: 0.2620
Epoch 1/4
1500/1500 [==============================] - 35s - loss: 0.5740 - acc: 0.8453 - val_loss: 1.8998 - val_acc: 0.3610
Epoch 2/4
1500/1500 [==============================] - 29s - loss: 0.4556 - acc: 0.8753 - val_loss: 2.0250 - val_acc: 0.2800
Epoch 3/4
1500/1500 [==============================] - 32s - loss: 0.3043 - acc: 0.9233 - val_loss: 2.1518 - val_acc: 0.2640
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 0.2228 - acc: 0.9440 - val_loss: 2.2786 - val_acc: 0.2240


Channel shift: randomly changing the R,G,B colors - 

In [42]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [43]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 36s - loss: 1.7021 - acc: 0.4867 - val_loss: 2.4090 - val_acc: 0.1870
Epoch 2/2
1500/1500 [==============================] - 26s - loss: 0.4589 - acc: 0.9033 - val_loss: 2.2991 - val_acc: 0.1870
Epoch 1/4
1500/1500 [==============================] - 36s - loss: 0.1476 - acc: 0.9873 - val_loss: 2.2256 - val_acc: 0.2770
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 0.0657 - acc: 0.9960 - val_loss: 2.3710 - val_acc: 0.3130
Epoch 3/4
1500/1500 [==============================] - 31s - loss: 0.0366 - acc: 0.9993 - val_loss: 2.4461 - val_acc: 0.2880
Epoch 4/4
1500/1500 [==============================] - 31s - loss: 0.0217 - acc: 1.0000 - val_loss: 2.4729 - val_acc: 0.2360


And finally, putting it all together!

In [44]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [45]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 35s - loss: 2.5215 - acc: 0.2167 - val_loss: 2.1978 - val_acc: 0.2070
Epoch 2/2
1500/1500 [==============================] - 29s - loss: 1.8185 - acc: 0.4020 - val_loss: 2.1768 - val_acc: 0.2200
Epoch 1/4
1500/1500 [==============================] - 36s - loss: 1.5109 - acc: 0.5073 - val_loss: 2.3769 - val_acc: 0.1280
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 1.3760 - acc: 0.5420 - val_loss: 2.5611 - val_acc: 0.1180
Epoch 3/4
1500/1500 [==============================] - 31s - loss: 1.2187 - acc: 0.6093 - val_loss: 2.7266 - val_acc: 0.1150
Epoch 4/4
1500/1500 [==============================] - 30s - loss: 1.1857 - acc: 0.6260 - val_loss: 3.0068 - val_acc: 0.1190


At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decisions.

In [46]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
1500/1500 [==============================] - 36s - loss: 1.0910 - acc: 0.6467 - val_loss: 3.0075 - val_acc: 0.1260
Epoch 2/5
1500/1500 [==============================] - 30s - loss: 0.9870 - acc: 0.6973 - val_loss: 3.0877 - val_acc: 0.1280
Epoch 3/5
1500/1500 [==============================] - 27s - loss: 0.9337 - acc: 0.7087 - val_loss: 2.8772 - val_acc: 0.1570
Epoch 4/5
1500/1500 [==============================] - 27s - loss: 0.8617 - acc: 0.7280 - val_loss: 2.7116 - val_acc: 0.1840
Epoch 5/5
1500/1500 [==============================] - 31s - loss: 0.8626 - acc: 0.7313 - val_loss: 2.8928 - val_acc: 0.1680


Lucky we tried that - we starting to make progress! Let's keep going.

In [47]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/25
1500/1500 [==============================] - 35s - loss: 0.8239 - acc: 0.7427 - val_loss: 2.5106 - val_acc: 0.2240
Epoch 2/25
1500/1500 [==============================] - 28s - loss: 0.7666 - acc: 0.7593 - val_loss: 2.4517 - val_acc: 0.2700
Epoch 3/25
1500/1500 [==============================] - 30s - loss: 0.7250 - acc: 0.7707 - val_loss: 2.1659 - val_acc: 0.3090
Epoch 4/25
1500/1500 [==============================] - 32s - loss: 0.6688 - acc: 0.7900 - val_loss: 1.9962 - val_acc: 0.3180
Epoch 5/25
1500/1500 [==============================] - 27s - loss: 0.6186 - acc: 0.8153 - val_loss: 1.9398 - val_acc: 0.3630
Epoch 6/25
1500/1500 [==============================] - 28s - loss: 0.6186 - acc: 0.8153 - val_loss: 1.8174 - val_acc: 0.4000
Epoch 7/25
1500/1500 [==============================] - 30s - loss: 0.6257 - acc: 0.8007 - val_loss: 1.7692 - val_acc: 0.3910
Epoch 8/25
1500/1500 [==============================] - 28s - loss: 0.5759 - acc: 0.8327 - val_loss: 1.7380 - val_acc:

Amazingly, using nothing but a small sample, a simple (not pre-trained) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our futher experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.